<h2>Do They Know?: A Brief Analysis of Accuracy of American Voters on Governmental Trivia Questions </h2>

<img src="../figs/title_photo.jpg"/>

In this analysis, we got hold of survey data conducted by the *American National Election Studies* data center. The surveys they conduct intend to capture how American citizens vote in each major election. But, rather than ask how they voted, today I'd like to know **who is voting**. More spceifically, I want to see how knowledgeable the voting public is on the machinations of the government they're electing. Does the knowledgeability differ across party lines or education? Let's find out!

In [1]:
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import scipy.stats as stats

<p> We'll be using data from ANES's *2012 Direct Democracy Study*. Below we can see how the data is structured<p>

In [11]:
data_path = Path("../data/anes_specialstudy_2012_directdem_dta/anes_specialstudy_2012_directdem_stata12.dta")
data = pd.read_stata(data_path)

print("Data dimensions: ", end="")
print(data.shape)
print("------------------------------")

print("Missing Values: ", end="")
print(data.isnull().sum().sum())
print("------------------------------")

print(data.iloc[0,:])

Data dimensions: (5415, 1037)
------------------------------
Missing Values: 0
------------------------------
dd_version                FINAL RELEASE, ANES 2012 Direct Democracy Stud...
dd_caseid                                                             20001
caseid                                                                 3001
weight_pre                                                           1.4708
weight_post                                                           1.518
                                                ...                        
post_ddpref_timing                                                       59
post_flawed_timing                                                       21
post_ddfavor_timing                                                      26
post_disp_close_timing                                                   13
post_qf1_timing                                                          30
Name: 0, Length: 1037, dtype: object


<p>Luckily we have no missing values! This does not mean we won't have to filter out non-answers, as we'll see later on. <p>

<p>We can also see that we have **5415** participants in the survey, and we have over **1000** columns corresponding to questions and information on the subject. This is a lot of information at our disposal, so we'll need to narrow down what we use. <p>